### Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import skimage.io as sk
from skimage import img_as_ubyte
from skimage.io import imread
from scipy import spatial
from tensorflow.keras.layers import Dense, Flatten, Input, Lambda, MaxPooling2D, Conv2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from PIL import Image
import cv2
import matplotlib.pyplot as plt

ImportError: Traceback (most recent call last):
  File "C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

### Displaying the of Image of Forge and Real Signature

In [ ]:
image1 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Real/00100001.png")
image2 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Real/00200002.png")
image3 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Real/00300003.png")
image4 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Real/00400004.png")
image5 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Real/00500005.png")

fig, ax = plt.subplots(1,5, figsize = (15,10))

ax[0].imshow(image1)
ax[0].set_title("Real_10")
ax[1].imshow(image2)
ax[1].set_title("Real_10")
ax[2].imshow(image3)
ax[2].set_title("Real_10")
ax[3].imshow(image4)
ax[3].set_title("Real_10")
ax[4].imshow(image5)
ax[4].set_title("Real_10")

image6 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Forge/02100001.png")
image7 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Forge/02100002.png")
image8 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Forge/02100003.png")
image9 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Forge/02100004.png")
image10 = sk.imread(r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train\Forge/02100005.png")

fig, ax1 = plt.subplots(1,5, figsize = (15,10))

ax1[0].imshow(image6)
ax1[0].set_title("Forge_10")
ax1[1].imshow(image7)
ax1[1].set_title("Forge_10")
ax1[2].imshow(image8)
ax1[2].set_title("Forge_10")
ax1[3].imshow(image9)
ax1[3].set_title("Forge_10")
ax1[4].imshow(image10)
ax1[4].set_title("Forge_10")

In [ ]:
train_path = r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train"
test_path = r"C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Test"

In [ ]:
Image_Width = 512
Image_Height = 512
Image_Size = (Image_Width, Image_Height)
Image_Channel = 3
batch_size=15

In [ ]:
model = Sequential()

## Conv layer 1
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(Image_Width,Image_Height, Image_Channel)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 2
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 3
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 4
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 5
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

## Conv layer 6
model.add(Conv2D(512, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.5, min_lr=0.00001)
callbacks = [early_stop, learning_rate_reduction]


In [ ]:
# Scaling all the images between 0 to 1 and applying Data Augmentation

train_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,)

In [ ]:
train_generator = train_datagen.flow_from_directory(r'C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Train',
                                              target_size=Image_Size,
                                              batch_size=32,
                                              class_mode = 'categorical')


In [ ]:
# Performing only scaling on the test dataset

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(r'C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Test',
                                                  target_size=Image_Size,
                                                  batch_size = 32,
                                                  class_mode='categorical')

In [ ]:
epochs = 10

history = model.fit_generator(train_generator,
                             epochs=epochs,
                             validation_data=test_generator,
                             validation_steps=len(test_generator),
                             steps_per_epoch=len(train_generator),
                             callbacks=callbacks)

## Plotting the Accuracy and Losses

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.legend()
plt.show()
plt.savefig('lossval_loss')

## Saving our model

In [ ]:
from tensorflow.keras.models import load_model

model.save(r'C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\ModelPickle\model.h5')

## Making our prediction with our model

In [ ]:
pred = model.predict(test_generator)
pred

In [ ]:
import numpy as np

pred = np.argmax(pred, axis=1)
pred

## Loading our model

In [ ]:
model = load_model(r'C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\ModelPickle\model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image

img = image.load_img(r'C:\Users\HARI PRASAD\PycharmProjects\SignatureForgery\venv\SignatureVerification\Dataset\Test\Forge/02103001.png', target_size=(512,512))

In [ ]:
x = image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x = x/255

from tensorflow.keras.applications.resnet50 import preprocess_input

x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
if(a==1):
    print("The signature is not fraud")
else:
    print("The signature is fraud")